In [1]:
from langchain.llms import CTransformers
from langchain import PromptTemplate
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


/Users/vaishnavishankardevadig/anaconda3/envs/mchat/lib/python3.8/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import os

In [3]:
os.environ['PINECONE_API_KEY']  = "0569ef24-684a-4511-81c7-972dedb13fd9"
os.environ['PINECONE_API_ENV']  = "gcp-starter"

In [4]:
def pdf_loader(data):
    l = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    doc = l.load()
    return doc

In [5]:
data_excerpts = pdf_loader("data/")

In [6]:
def split_excerpt(data_ex):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    chunks = splitter.split_documents(data_ex)
    return chunks

In [7]:
chunks = split_excerpt(data_excerpts)

In [8]:
chunks

[Document(page_content='https://allebookstores.com', metadata={'source': 'data/current-medical-diagnosis-and-treatment-2024_compressed_compressed.pdf', 'page': 0}),
 Document(page_content='CMDT2024_FM_pi-xx.indd   1 13/07/23   2:47 PMhttps://allebookstores.com', metadata={'source': 'data/current-medical-diagnosis-and-treatment-2024_compressed_compressed.pdf', 'page': 1}),
 Document(page_content='CMDT2024_FM_pi-xx.indd   2 13/07/23   2:47 PMhttps://allebookstores.com', metadata={'source': 'data/current-medical-diagnosis-and-treatment-2024_compressed_compressed.pdf', 'page': 2}),
 Document(page_content='2024\nCURRENT\nMedical Diagnosis  \n& Treatment\nSIXTY-THIRD EDITION\nEdited by\nNew York  Chicago  San Francisco  Athens  London  Madrid  Mexico City  \nMilan  New Delhi  Singapore  Sydney  Torontoa LANGE medical book\nMaxine A. Papadakis, MD \nProfessor of Medicine, Emeritus \nDepartment of Medicine\nUniversity of California, San Francisco\nStephen J. McPhee, MD\nProfessor of Medicine, 

In [9]:
len(chunks)

22602

In [10]:
def hfembedmodel():
    emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-miniLM-L6-v2")
    return emb

In [11]:
embed = hfembedmodel()

In [12]:
res = embed.embed_query('Disha')

In [13]:
len(res)

384

In [29]:
!pip install pinecone-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [36]:
pip install -U langchain-pinecone

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached jsonpointer-2.4-py2.py3-none-any.whl.metadata (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 6.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 11.5 MB/s eta 0:00:00
Using cached jsonpointer-2.4-py2.py3-none-any.whl (7.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 17.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [72]:
from langchain_pinecone import PineconeVectorStore

In [73]:
pinecone.Pinecone(api_key = os.environ['PINECONE_API_KEY'], environment= os.environ['PINECONE_API_ENV'])
index_name = "mchat"


In [75]:
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 3.8 MB/s eta 0:00:0000:0100:01


In [26]:
from langchain.vectorstores import FAISS
ds = FAISS.from_documents(chunks, embed)

In [28]:
prompt_template = """
Use the following piece of information to answer the user's question.
If you do not know the answer, just say you do not know, do not try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer: 
"""


In [29]:
p = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chaintypekwargs = {"prompt": p}

In [30]:
model = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
model_type ="llama",
config={'max_new_tokens' :512,
'temperature': 0.83})

In [31]:
quest = RetrievalQA.from_chain_type(llm = model, 
chain_type="stuff", 
retriever= ds.as_retriever(search_kwargs={'k': 2}),
return_source_documents = True, 
chain_type_kwargs=chaintypekwargs)

#asretriever(search_kwargs={'k': 2}), 

In [34]:
while True:
    ip = input("Input: ")  
    result = quest({"query": ip})
    print("Answer: ", result["result"])


Answer:  Bronchitis is an inflammation of the bronchial tubes, which are the airways that lead to the lungs. There are two main types of bronchitis: acute and chronic. Acute bronchitis is usually caused by a viral infection and can be accompanied by a dry cough, chest tightness, and shortness of breath. Chronic bronchitis, on the other hand, is a long-term condition that can be caused by exposure to pollutants such as cigarette smoke or dust. It can lead to persistent coughing and difficulty breathing. In some cases, bronchitis can also cause complications such asions such as such asions such asions such asions such asions such asions such asions such asions such asions such asations such asions such asions such as-lications such asions such asions such as such as -lications such as mentioned inations such asionations such asions such asions such asions such asions such asions such asions such asinsations such asions such asions such asions such as.


In [90]:
print(type(quest))



<class 'langchain.chains.retrieval_qa.base.RetrievalQA'>
